In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [ ]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/known/*.feather"))
#test1_filenames = building1_filenames - training_filenames

In [ ]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [ ]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)

#training_data.head()

orientation_acc = training_data[["iphoneAccX", "iphoneAccY", "iphoneAccZ"]]
orientation_gyro = training_data[[ "iphoneGyroX", "iphoneGyroY", "iphoneGyroZ"]]
orientation_mag = training_data[[ "iphoneMagX", "iphoneMagY", "iphoneMagZ"]] 

#orientation_data.head()

In [ ]:
orientation_target = training_data[["orientX", "orientY", "orientZ", "orientW"]] 
orientation_target.head()

In [ ]:
acc_np = orientation_acc.to_numpy()
gyro_np = orientation_gyro.to_numpy()
mag_np = orientation_mag.to_numpy()

In [ ]:
orient_np = orientation_target.to_numpy()
len(orient_np)

In [ ]:
def generate_training_samples(acc_, gyro_, mag_, orient_, batch_size = 64):
    while True:
        xa_batch = np.zeros((batch_size,100,3))
        xg_batch = np.zeros((batch_size,100,3))
        xm_batch = np.zeros((batch_size,100,3))
        y_theta_batch = np.zeros((batch_size,4))
        y_sigma_batch = np.finfo(np.float32).eps* np.ones((batch_size,6)) 
        y_batch = np.concatenate((y_theta_batch, y_sigma_batch), axis=1)
        #print(y_batch.shape)
        
        j = 0
        for i in range(len(orient_)):
            xa_batch[j,:,:] = acc_[i, :]
            xg_batch[j,:,:] = gyro_[i,:]
            xm_batch[j,:,:] = mag_[i,:]
            y_batch[j,:4] = orient_[i,:]
            j += 1
            if j >= batch_size:
                j = 0              
                yield([xa_batch,xg_batch, xm_batch],[y_batch])
            #---------------------------------------------------------------------------------------

In [ ]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_OrientNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_.hdf5') # vloss_{val_loss:.4f}
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [ ]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [ ]:

def quaterinion_inv(q):
    return (1/(np.sum(q**2))) * np.array([-1*q[0], -1*q[1], -1*q[2], q[3]])

def log_bar(q):
    if q[3] == 0:
        return np.array([0, 0, 0])
    else:
        v = q[:3]
        return np.arctan2(np.norm(v), q[3])/np.norm(v) * v


def quaternion_multiply(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

def boximinus(q1, q2):
    return 2 * log_bar(quaternion_multiply(quaterinion_inv(q2), q1))

In [ ]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    # q = y[:4] sig = y[4:]
    q = y_true[:4]
    q_est = y_pred[:4]

    delta = boximinus(q, q_est)

    return tf.convert_to_tensor(np.transpose(delta) @ delta)

class MyLossP(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # y_pred = tf.convert_to_tensor_v2(y_pred)
    # y_true = tf.cast(y_true, y_pred.dtype)
    return tf.sqrt(tf.reduce_mean((y_pred - y_true)**2, axis=-1))

class MyLossA(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # y_pred = tf.convert_to_tensor_v2(y_pred)
    # y_true = tf.cast(y_true, y_pred.dtype)
    
    return tf.sqrt(tf.reduce_mean((y_pred - y_true)**2, axis=-1))



In [ ]:
class OreintNet():
    def build_model(self):
        # Building Network
        #1. Define inputs
        Acc_input = Input(shape=(100,3), batch_size=64 , name = 'Acc_input')    
        Gyro_input = Input(shape=(100,3),batch_size=64   , name = 'Gyro_input')
        Mag_input = Input(shape=(100,3), batch_size=64   , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        #
        ##MergedLayer = Input(shape=(9) , name = 'imu_input')
        #2. LSTM
        LSTM1 = (LSTM(100, return_sequences=True))(MergedLayer) # , return_sequences=True
        LSTM2 = (LSTM(100, return_sequences=False))(LSTM1)
        #print("Output shape of the LSTM layer's output : ", LSTM2.shape)
        
        #3. Fully-Connected (Sigma)
        Dense1 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2)
        Dense2 = Dense(units=32, activation='tanh')(Dense1)
        theta_output = Dense(units=4, name='theta_out')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Theta)
        Dense3 = Dense(units=100, input_shape=(64,100), activation='tanh')(LSTM2) # input_shape=(
        Dense4 = Dense(units=32, activation='tanh')(Dense3)
        sigma_output = Dense(units=6, name='sig_out')(Dense4) # Theta Outputs  activation='linear'

        ### specifies the weight per loss
        ##losses = {
        ##"theta_out": MyLossP(),
        ##"sig_out": MyLossA(),
        ##}
        ##lossWeights = {"theta_out": 0.5, "sig_out": 0.5}

        output = Concatenate()([theta_output, sigma_output])
        #5. Define and compile The model
        Network = Model([Acc_input,Gyro_input, Mag_input],  [output])  #  [Acc_input,Gyro_input, Mag_input]
        Network.compile(loss=OreintLoss(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['mape']) #loss=OreintLoss()
        return Network
        

In [ ]:
#class OreintNet(Model):
    

In [ ]:
network_builder = OreintNet()
orientation_network = network_builder.build_model()

In [ ]:
orientation_network.summary()

In [ ]:
Nepochs = 20
history = orientation_network.fit(
    generate_training_samples(acc_np, gyro_np, mag_np, orient_np),
    epochs=Nepochs,
    initial_epoch=initial_epoch,
    steps_per_epoch=len(orient_np),
    batch_size=64,
    callbacks = [check_point_callback]
)

In [ ]:
pos_learning_rate = 0.001